# 武汉市共享单车出行报告—CityPhi展示
by武汉市交通发展战略研究院 郑猛/zmfly@163.com / 13018035815

In [1]:
# coding: utf-8

In [2]:
#加载基础模块
import pandas as pd
import numpy as np
import os, math, copy
import time as _time
import sys, random

In [3]:
# start
import cityphi.application
pd.options.display.max_rows = 15
%matplotlib inline
import matplotlib.pyplot as plt


%gui cityphi
app = cityphi.application.Application()

import colorbrewer
#RGB color
Black = (0,0,0)
White = (255,255,255)
Red = (255,0,0)
Lime = (0,255,0)
Blue = (0,0,255)
Yellow = (255,255,0)
Cyan = (0,255,255)
Magenta = (255,0,255)
Silver = (192,192,192)
Gray = (128,128,128)
Maroon = (128,0,0)
Olive = (128,128,0)
Green = (0,128,0)
Purple= (128,0,128)
Teal = (0,128,128)
Navy = (0,0,128)
Grad2 = (150, 150, 150)
LightYellow = (245, 245, 80)

from cityphi.feature import *
from cityphi.layer import *
import cityphi.attribute
from cityphi.attribute import DiscreteColorAttribute, FeatureAttribute
from cityphi.attribute import Attribute, MultiplierAttribute

import cityphi.feature as _feat
import cityphi.layer as _layer
import cityphi.parameter as _param
import cityphi.attribute as _att
import cityphi.query as _query
import cityphi.data as _data

from IPython.display import display, HTML
from ipywidgets import widgets
from ipywidgets import interact

import types
import fiona.crs as _crs
import pyproj as _proj

src_p = _proj.Proj(_crs.from_epsg(4326))  # lat/lon (WGS84)
dst_p = _proj.Proj(_crs.from_epsg(3857))

def get_time(t):
    hh = int(t/3600)
    mm = int((t-hh*3600)/60)
    ss = int(t-hh*3600-mm*60)
    return "%02d:%02d:%02d" % (hh, mm, ss)

import cityphi.widget as _widget
map_layer = app.layers[0]
map_layer.name = u"地图"

In [4]:
# HUD
class CountWidget(_widget.Widget):
    _js_type = "count_widget"
    
    def __init__(self, label, value, max_value, fill_color='#FF0000'):
        super(CountWidget, self).__init__()
        self.label = label
        self.value = value
        self.max_value = max_value
        self.fill_color = fill_color
    label = _param.StringParameter()
    value = _param.IntegerParameter(0)
    max_value = _param.IntegerParameter(0)
    fill_color = _param.ColorParameter()

map_layer = app.layers[0]
map_layer.name = u"地图"

In [5]:
#加载基础模块
import pandas as pd
import numpy as np
import os, math, copy
import time as _time
import sys, random

#如果没有第一行，则图形文件会从新窗口弹出，程序运行亦会暂停
%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams['font.sans-serif']=['SimHei'] #用来正常显示中文标签
plt.rcParams['axes.unicode_minus']=False #用来正常显示负号

pd.options.display.max_rows = 16 #最多显示16行

In [6]:
#cityphi相关
import cityphi.application
from cityphi.feature import *
from cityphi.layer import *
import cityphi.attribute
from cityphi.attribute import DiscreteColorAttribute, FeatureAttribute
from cityphi.attribute import Attribute

import cityphi.feature as _feat
import cityphi.layer as _layer
import cityphi.data as _data
import cityphi.parameter as _param

# widgets 播放器，插件
from IPython.display import display, HTML
from ipywidgets import widgets
from ipywidgets import interact
import types

# Fiona地理数据引擎Python库
import fiona.crs as _crs
import pyproj as _proj
src_p = _proj.Proj(_crs.from_epsg(4326))  # lat/lon (WGS84)  longitude经度 latitude维度 
dst_p = _proj.Proj(_crs.from_epsg(3857))
#from basics import src_p_utm50N, src_p, dst_p    #如果此条打开，则上面两条关闭
#get_ipython().magic(u'matplotlib inline')

In [7]:
# start CityPhi
#运行app 则cityphi窗口弹出
%gui cityphi
app = cityphi.application.Application()
# add a custom base map
web_basemap = app.layers[0]
import cityphi.core.layer.rastertilelayer as _rastertilelayer
import cityphi.core._core as _core
url_template = "http://webrd02.is.autonavi.com/appmaptile?lang=zh_cn&size=1&scale=1&style=7&x=%2&y=%3&z=%1"  #高德地图
custom_map_feat = _core.RasterTileWebFeature(url_template, 18)
web_basemap.feature = custom_map_feat
web_basemap.name = u"高德地图"
web_basemap.opacity = 30

In [8]:
# 数据目录 （data directory）
data_path = "./Data"
work_path = "./Work"
gis_path = "./GISData"
#os.getcwd() #获取当前工作目录

## 加载路网

In [9]:
#-加载武汉现状路网
whroad_shapefile = os.path.join(gis_path, "highway_line.shp")
whroad_feature, whroad_attributes = _data.import_shapefile(whroad_shapefile)
#
whroad_layer = _layer.PolylineLayer(whroad_feature,max_vertices=200000)
app.add_layer(whroad_layer)
whroad_layer.name = u"武汉现状路网"
whroad_layer.width = 5.0
whroad_layer.color = (255,255,255)
#whroad_layer.priority = 2

# 1 订单数据

## 1.1 租还车点位集合

In [10]:
#加载原始订单数据
mobikefile= os.path.join(data_path,"order_sample.csv")
mobike_df_raw = pd.read_csv(mobikefile)
print mobike_df_raw.shape
mobike_df_raw[:5]

(27384, 9)


,orderid,userid,bikeid,starttime,startpositionx,startpositiony,endtime,endpositionx,endpositiony
0,2112548,6754,144920,2017-06-23 20:00:24.0,114.206620,30.607073,2017-06-23 20:09:48.0,114.20209987,30.61168888
1,2355538,101224,153238,2017-06-23 20:00:28.0,114.039315,30.588055,2017-06-23 20:16:33.0,114.03877515,30.56942600
2,2761314,139889,166490,2017-06-23 20:00:27.0,114.236080,30.507850,2017-06-23 20:10:43.0,114.22171173,30.51030346
3,2980580,799957,173744,2017-06-23 20:00:25.0,114.303945,30.548364,2017-06-23 20:05:25.0,114.30200972,30.54576367
4,3009149,719156,174686,2017-06-23 20:00:24.0,114.329828,30.504223,2017-06-23 20:05:27.0,114.33079898,30.50466843


In [11]:
#租还车点位集合
mobike_df_raw=mobike_df_raw[mobike_df_raw['endpositionx']!='null']
grid_O = np.column_stack(
    (mobike_df_raw.startpositionx.values,mobike_df_raw.startpositiony.values))
grid_D = np.column_stack(
    (mobike_df_raw.endpositionx.values,mobike_df_raw.endpositiony.values))
grid_OD = np.row_stack((grid_O, grid_D))
grid_week = pd.DataFrame(grid_OD, columns=['Lng','Lat'])
grid_week = grid_week.drop_duplicates()    #基于全部列去重
#grid_week = grid_week.sort_values(by=[ 'Lng','Lat']).reset_index(drop=True)
grid_week = grid_week.sort(['Lng','Lat']).reset_index(drop=True)
grid_week['trackid']=grid_week.index+1
print grid_week.shape
grid_week.head()

(54350, 3)


,Lng,Lat,trackid
0,104.1176,30.79568,1
1,113.233,23.12879,2
2,113.4343,30.35669,3
3,113.4507,30.37376,4
4,113.4553,30.3669,5


In [12]:
#坐标可视化-高德坐标
week_x, week_y =  _proj.transform(src_p, dst_p,grid_week.Lng.values, grid_week.Lat.values)
week_points = np.column_stack(
    (week_x, week_y, np.zeros(len(grid_week))))
#grid_ids = grid_mobike.index.values
week_ids = grid_week.trackid.values
gridweek_feature = PointFeature(week_ids, week_points)
gridweek_layer = PointLayer(gridweek_feature)
gridweek_layer.min_pixel_size = 2
gridweek_layer.radius = 2
gridweek_layer.color = (0, 227, 252)
app.add_layer(gridweek_layer)
app.set_view(app.full_view)
#vehicle_layer.color.value = Blue
gridweek_layer.name = u"单车租还车位置集"
#app.background_color = (25, 25, 25)  # make the background black

## 1.2 特征日分析

In [13]:
#bike_od_df_arr_raw = pd.read_csv(os.path.join(work_path,'33_bike_od_20170620.csv'))  #完整订单OD数据
bike_od_df_arr = pd.read_csv(os.path.join(work_path,'mobike_od_20170620_2arr.csv'))  #完整订单OD数据
print bike_od_df_arr.shape
bike_od_df_arr[:5]

(16270, 10)


,Orderid,Userid,Bikeid,TripID,Time,Lat,Lng,Timeuse_OD,Len_OD,IO
0,73,771130,41486,0,71461,39.903209,116.249230,368,600.626277,0
1,73,771130,41486,0,71829,39.906070,116.243267,368,600.626277,1
2,515,588639,41689,1,38189,30.573193,114.279657,197,373.321695,0
3,515,588639,41689,1,38386,30.570871,114.276847,197,373.321695,1
4,1063,1161549,41718,2,38192,30.362188,114.322422,980,1100.534766,0


In [14]:
#坐标可视化-高德坐标
day_x, day_y =  _proj.transform(src_p, dst_p,
                                  bike_od_df_arr['Lng'].values, bike_od_df_arr['Lat'].values)
day_points = np.column_stack(
    (day_x, day_y, np.zeros(len(bike_od_df_arr))))
day_ids = bike_od_df_arr.index.values
#day_ids = bike_od_df_arr.SegID.values
gridday_feature = PointFeature(day_ids, day_points)
gridday_layer = PointLayer(gridday_feature)
gridday_layer.min_pixel_size = 2
gridday_layer.radius = 2
gridday_layer.color = (0, 227, 252)  #(255,71,19)  #摩拜橙色   (111, 173, 246)天蓝色
app.add_layer(gridday_layer)
app.set_view(app.full_view)
#vehicle_layer.color.value = Blue
gridday_layer.name = u"摩拜单车租还车位置集（6月20）"
app.background_color = (24, 30, 46)  #   深蓝色
#app.background_color = (25, 25, 25)  # make the background black

In [15]:
bike_od_df_arr['Endtime']=bike_od_df_arr.Time.values + bike_od_df_arr.Timeuse_OD.values
bike_od_df_arr['Speed']=bike_od_df_arr.Len_OD.values/bike_od_df_arr.Timeuse_OD.values*3.6  

In [16]:
gridday_feature.add_attribute("IO", "int32",
                        bike_od_df_arr.TripID.values, bike_od_df_arr.IO.values)
gridday_feature.add_attribute("Time", "int32", 
                              bike_od_df_arr.TripID.values, bike_od_df_arr.Time.values)
gridday_feature.add_attribute("end_time", "int32",
                              bike_od_df_arr.TripID.values, bike_od_df_arr.Endtime.values)
gridday_feature.add_attribute("Bikeid", "int32",
                        bike_od_df_arr.TripID.values, bike_od_df_arr.Bikeid.values)
gridday_feature.add_attribute("Userid", "int32",
                        bike_od_df_arr.TripID.values, bike_od_df_arr.Userid.values)
gridday_feature.add_attribute("Orderid", "int32",
                        bike_od_df_arr.TripID.values, bike_od_df_arr.Orderid.values)

Attribute(Orderid)

## 1.3 特征日点位聚集

In [17]:
#设置网格聚集
#point_ids = bike_od_df_arr.trackid.values  
point_ids = bike_od_df_arr.index.values
grid_feature = PointFeature.from_points(point_ids, day_ids, gridday_feature)
#grid_feature = PointFeature(point_ids, track_points)
point_grid_layer = PointLayer(grid_feature, max_vertices=200000)
app.add_layer(point_grid_layer)
point_grid_layer.name = u'点网格集聚'
point_grid_layer.stacked = True
grid_feature.add_attribute("start_time", "int32", 
                           bike_od_df_arr.TripID.values, bike_od_df_arr.Time.values)
grid_feature.add_attribute("end_time", "int32", 
                           bike_od_df_arr.TripID.values, bike_od_df_arr.Endtime.values)
point_grid_layer.start_time = FeatureAttribute("start_time")
#point_grid_layer.end_time = FeatureAttribute("end_time")
point_grid_layer.end_time = FeatureAttribute("start_time")

In [18]:
# time slider. Run this only once
import cityphi.widget as _widget

def get_time(t):
    hh = int(t/3600)
    mm = int((t-hh*3600)/60)
    ss = int(t-hh*3600-mm*60)
    return "%02d:%02d:%02d" % (hh, mm, ss)

html_widget = _widget.HTMLWidget("")
app.add_widget(html_widget)

def format_timestamp(t):
    return """
    <span style='font-size: 20px; line-height: 20px'>
    %s
    </span>""" % (get_time(t))

In [19]:
def format_timestamp(t):
    return """
    <span style='font-size: 30px; line-height: 30px'>
    %s
    </span>""" % (get_time(t))

In [20]:
#刷新时间
def change_time(t):
    html_widget.html = format_timestamp(t)
    point_grid_layer.time = t
    point_grid_layer.time_window = t - 3600, t  # 1 小时集聚 
    
time_slider = _widget.TimeSlider(0 * 3600, 24 * 3600, get_time, change_time)
app.add_widget(time_slider) 

In [21]:
# set size, height, color
start_value = 1
end_value = 5
start_color =(255,255,0) #(255,255,0)
end_color = (255,0,0)  #摩拜橙色  (255,71,19)
level =19            # set level: 1-21， 网格大小一共有 21 级  18
#
point_grid_layer.min_pixel_size = 5
point_grid_layer.height =1
#
point_grid_layer.color = cityphi.attribute.LinearInterpolatedColorAttribute(
    start_value, end_value, start_color, end_color, cityphi.attribute.StackedAttribute())
point_grid_layer.feature = \
    grid_feature.aggregate_grid(level) #grid size max.= 21

## 1.4 特征日运行期望线

In [22]:
#出行期望线
day_x, day_y =  _proj.transform(src_p, dst_p,bike_od_df_arr['Lng'].values, bike_od_df_arr['Lat'].values)
day_points = np.column_stack((day_x, day_y, np.zeros(len(bike_od_df_arr))))
bike_trip_ids = bike_od_df_arr.TripID.values
#bike_dl_grid_ids = bike_od_df_arr.trackid.values
bike_dl_times = bike_od_df_arr.Time.values
bike_dl_feature = TrajectoryFeature(bike_trip_ids, day_points, bike_dl_times)
#bike_dl_grid_ids = bike_od_df_arr.index.values
#bike_dl_feature = TrajectoryFeature.from_points(bike_trip_ids, bike_dl_grid_ids, 
#                                                 bike_dl_times, gridday_feature)
bike_dl_layer = _layer.ParabolicTrajectoryLayer(bike_dl_feature, max_vertices=200000)
app.add_layer(bike_dl_layer)
bike_dl_layer.name = u'摩拜单车出行期望线'
bike_dl_layer.height = 500
bike_dl_layer.color.value = (0, 247, 255) 
bike_dl_layer.min_pixel_size = 2
bike_dl_layer.width = 20
#bike_dl_layer.visible = True
#bike_dl_layer.hide()
#bike_dl_feature.add_attribute("card_no", "string", 

In [23]:
# time slider. Run this only once
import cityphi.widget as _widget

def get_time(t):
    hh = int(t/3600)
    mm = int((t-hh*3600)/60)
    ss = int(t-hh*3600-mm*60)
    return "%02d:%02d:%02d" % (hh, mm, ss)

html_widget = _widget.HTMLWidget("")
app.add_widget(html_widget)

def format_timestamp(t):
    return """
    <span style='font-size: 20px; line-height: 20px'>
    %s
    </span>""" % (get_time(t))

In [24]:
# update time slider
def change_time(t):
    # ...
    html_widget.html = format_timestamp(t)
    bike_dl_layer.time = t                          
    bike_dl_layer.time_window = t - 10*60, t  #length of tail in second
    #point_grid_layer.time_window = t - 3600, t  # 1 小时集聚 
    #point_grid_layer.time_window = 0, t  # 1 小时集聚     
time_slider = _widget.TimeSlider(0 * 3600, 24 * 3600, get_time, change_time)
app.add_widget(time_slider) 
#time_slider.time_callback = change_time

## 1.5 运行水滴图

In [25]:
#出行水滴图
bike_path_ayer = _layer.TrajectoryLayer(bike_dl_feature, max_vertices=200000)
app.add_layer(bike_path_ayer)
bike_path_ayer.name = u'摩拜单车出行路径'
bike_path_ayer.color.value = (168, 103, 48) #橘色 类似灯光的效果
bike_path_ayer.min_pixel_size = 4
bike_path_ayer.width = 15

In [26]:
#设置颜色
bike_path_ayer.color.is_time_varying=True
speed_att =  cityphi.attribute.MultiplierAttribute(3.6,  cityphi.attribute.FeatureAttribute("speed"))
speed_att.is_time_varying = True
#bike_dl_layer.color.attribute.display_name=u'speed'
color_breaks =[0.0,3.0, 6.0, 10,999.0]
#colors=[(88, 249, 0), (252, 166, 0), (255, 0, 0), (151, 151, 151)]   #（参照手机信令样式）
colors=[(88, 249, 0), (252, 166, 0), (249, 33, 204), (249, 0, 0)]
color_labels=[u'停止',u'慢速', u'中速', u'快速', u'高速']
bike_path_ayer.color = cityphi.attribute.ContinuousColorAttribute(colors, 
        color_breaks, speed_att, color_labels)

In [27]:
#刷新时间
def change_time(t):
    html_widget.html = format_timestamp(t)
    bike_dl_layer.time = t                          
    bike_dl_layer.time_window = t - 10*60, t  #length of tail in second
    #point_grid_layer.time_window = t - 3600, t  # 1 小时集聚
    bike_path_ayer.time_window = t - 5.*60, t  #length of tail in second
time_slider.time_callback = change_time

# 2 轨迹数据

## 2.1 租还车点位集合

In [28]:
#加载轨迹数据
trackfile= os.path.join(work_path,"2_mobike_track_2017-06-20.csv")
track_df = pd.read_csv(trackfile)
print track_df.shape
track_df[:5]

(1040, 6)


,Orderid,Userid,Bikeid,Time,Lng,Lat
0,1274,82151654,1483949,30216,114.282150,30.587905
1,1274,82151654,1483949,30845,114.289068,30.586030
2,1274,82151654,1483949,30870,114.289147,30.584945
3,1702,96916278,1511545,27720,114.252881,30.579861
4,1702,96916278,1511545,28264,114.261973,30.578904


In [29]:
#坐标可视化-高德坐标
track_x, track_y =  _proj.transform(src_p, dst_p,track_df.Lng.values, track_df.Lat.values)
track_points = np.column_stack(
    (track_x, track_y, np.zeros(len(track_df))))
track_ids = track_df.index.values
#track_ids = track_df.trackid.values
gridtrack_feature = PointFeature(track_ids, track_points)
gridtrack_layer = PointLayer(gridtrack_feature)
gridtrack_layer.min_pixel_size = 4
gridtrack_layer.radius = 2
gridtrack_layer.color = (0, 255, 255)
app.add_layer(gridtrack_layer)
app.set_view(app.full_view)
#vehicle_layer.color.value = Blue
gridtrack_layer.name = u"单车运行轨迹"
#app.background_color = (25, 25, 25)  # make the background black

In [30]:
mobike_df = pd.read_csv(os.path.join(work_path,"4_bike_trips_20170620_cityphi.csv"))
# suppose, a vehicle stay in the grid for 30 seconds after sending GPS signal
mobike_df['EndTime'] = mobike_df.Time + 30
mobike_df[:3]

,Orderid,TripID,SegID,Time,Speed,X,Y,Color_Ind,EndTime
0,15361,1,1,80426,8.105772,114.347315,30.581608,2,80456
1,15361,1,2,81260,23.255602,114.327992,30.578814,5,81290
2,15361,1,3,81262,0.000000,114.327859,30.578789,0,81292


In [31]:
# Mobike trajectory layer
bike_x, bike_y =  _proj.transform(
    src_p, dst_p, 
    mobike_df['X'].values, 
    mobike_df['Y'].values)
track_points = np.column_stack((bike_x, bike_y, np.zeros(len(mobike_df))))
track_trip_ids = mobike_df.SegID.values   
track_times = mobike_df.Time.values
track_feature = TrajectoryFeature(track_trip_ids, track_points, track_times)
track_layer = AnimatedPointLayer(track_feature, time_slice=60 * 60)
track_layer.min_pixel_size = 8.0
#vehicle_layer.radius = 2
track_layer.color = (250,0, 0)
#vehicle_layer.color.value = Blue
track_layer.name = u"摩拜单车运行轨迹"
app.add_layer(track_layer)
app.set_view(app.full_view)

In [32]:
# bike path layer
path_layer = TrajectoryLayer(track_feature, time_slice=60*60 )
path_layer.width = 10
path_layer.min_pixel_size = 4
path_layer.color = track_layer.color
path_layer.glowing = True
path_layer.name = u"摩拜单车出行路径"
app.add_layer(path_layer)
app.background_color = (25, 25, 25)  # make the background black

In [33]:
# speed color
Gray = (128,128,128)
red = (255,0,0)
dark_orange = (255,140,0)
green_yeallow = (173,255,47)
Green = (0,128,0)
trip_colors = [Gray, red,dark_orange, green_yeallow, Green]
color_labels = [u'速度0',u'速度0-5',u'速度5-10',u'速度10-15',u'速度15-20']
track_layer.color = DiscreteColorAttribute(trip_colors, 
                        FeatureAttribute("type_color"),color_labels)
track_layer.color

In [34]:
# use speed computed by CityPhi. This can be used for raw data without pre. process.
use_customer_color = True
if not use_customer_color:
    class SpeedColor(Attribute):    
        def load(self, feature_data):
            return 1 + feature_data.speed * 3.6 // 10
    color_labels = [u'速度0',u'速度0-5',u'速度5-10',u'速度10-15',u'速度15-20']
    track_layer.color = DiscreteColorAttribute(trip_colors, SpeedColor(),color_labels,
                                              priorities=[0, 1, 2, 3, 4, 5])
    track_layer.color.is_time_varying = True
    path_layer.color = track_layer.color    
    #track_layer.color

In [35]:
path_layer.color = track_layer.color

In [36]:
# time slider. Run this only once
import cityphi.widget as _widget

def get_time(t):
    hh = int(t/3600)
    mm = int((t-hh*3600)/60)
    ss = int(t-hh*3600-mm*60)
    return "%02d:%02d:%02d" % (hh, mm, ss)

html_widget = _widget.HTMLWidget("")
app.add_widget(html_widget)

def format_timestamp(t):
    return """
    <span style='font-size: 40px; line-height: 40px'>
    %s
    </span>""" % (get_time(t))

def change_time(t):
    html_widget.html = format_timestamp(t)
    track_layer.time = t                          
    path_layer.time_window = t - 0.05*3600, t  #length of tail in second
    
time_slider = _widget.TimeSlider(0 * 3600, 24 * 3600, get_time, change_time)
app.add_widget(time_slider)

In [37]:
#刷新时间
def change_time(t):
    html_widget.html = format_timestamp(t)
    track_layer.time = t                          
    #path_layer.time_window = t - 0.05*3600, t  #length of tail in second
    
time_slider.time_callback = change_time

## 查核线分析

In [38]:
#Screenline (60 min resolution) for the whole day  运行之前先在屏幕设定查核线
resolution = 60*60
in_results = []
out_results = []
x_values = []
total_queries = int((time_slider.end_time - time_slider.start_time) / resolution)
for i in range(total_queries):
    start_time = time_slider.start_time + resolution * i
    end_time = start_time + resolution
    result = track_layer.query_screenline(
        app.selection, (start_time, end_time))
    in_results.append(len(result[0]))  #upward
    out_results.append(len(result[1]))
    #hh=end_time/3600
    #x_values.append(end_time)  #秒
    #x_values.append(hh)         #小时，带有小数
    x_values.append(get_time(end_time)[:2])

in_results_array = np.array(in_results)
out_results_array = np.array(out_results)
time_array=np.array(x_values)
#画图
screen_volau=np.column_stack((in_results_array, out_results_array))
screen_df=pd.DataFrame(screen_volau, index=time_array,columns=['volau_in','volau_out'])
screen_df.head()

AttributeError: 'NoneType' object has no attribute 'update'

In [ ]:
#画图
screen_df.plot(title="Yangtse_River Screen_line Volaus",figsize=(30, 10),kind="bar")  #linewidth=2.5

In [ ]:
#流量汇总
screen_df.apply(lambda x: x.sum())     #行求和

In [ ]:
#输出到外部文件
screen_df['volau_sum'] = screen_df.apply(lambda x: x.sum(), axis=1)  #列求和
screen_df.loc['Row_sum'] = screen_df.apply(lambda x: x.sum())     #行求和
screen_df.to_csv(os.path.join(work_path,"09_20170620_Cross_River.csv"))  #不输出索引列 ,index=False